In [ ]:
# --- Cell 0: Run PyPetaKit5D Processing (Interactive) ---
import opym
from pathlib import Path
import importlib
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

# Reload the opym module to make sure we have the new functions
try:
    importlib.reload(opym.petakit)
except AttributeError:
    # Handle first-time load if 'petakit' isn't yet on the module
    pass
importlib.reload(opym)


# --- Create Interactive Widgets ---
print("1. Select the base data folder (e.g., '.../cell')")
print("2. Click 'Run PetaKit Processing'")

# This chooser is set to select directories
dir_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    title="<b>Select Base Data Folder (e.g., '.../cell'):</b>",
    show_only_dirs=True
)

run_button = widgets.Button(
    description="Run PetaKit Processing", button_style="success"
)
run_output = widgets.Output()

# --- Define the function to run on button click ---
def on_run_button_clicked(b):
    with run_output:
        run_output.clear_output()
        
        if not dir_chooser.value:
            print("❌ ERROR: No directory selected. Please select a folder and try again.")
            return

        DATA_FOLDER_PATH = Path(dir_chooser.value)

        try:
            print(f"--- Running PetaKit5D processing for: {DATA_FOLDER_PATH.name} ---")
            
            # Call the function from your package
            opym.run_petakit_processing(
                DATA_FOLDER_PATH,
                # You can override any default parameter here, for example:
                # sheet_angle_deg=21.8,
                # interp_method="cubic"
            )
            
            print("\n✅ PetaKit5D job finished!")

        except FileNotFoundError as e:
            print(f"\n❌ ERROR: Could not find required directories or files.")
            print(f"  Details: {e}")
            print("  Please ensure you have run the 'opym.run_processing_job' step first.")
        except Exception as e:
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)

# --- Link button and display UI ---
run_button.on_click(on_run_button_clicked)
display(widgets.VBox([dir_chooser, run_button, run_output]))